# 기상청 컬럼 생성

* AVRG_TMPRT : 평균기온(°C)	
* DAY_RAINQTY : 강수량(mm)
* DAY_MSNF : 적설량(cm)
* AVRG_WS : 평균 풍속(m/s)
* AVRG_HUMIDITY : 평균 습도(%)

## 기상정보 불러오기

In [ ]:
# 기상 정보 : https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36

# 일별 기상정보 받아오기 
path = "/content/drive/MyDrive/Competitions/a firefighting competitions/기상 데이터/day_climate.csv"
climate_data_day = pd.read_csv(path, encoding = 'cp949')

# 시간별 기상정보 받아오기
path = "/content/drive/MyDrive/Competitions/a firefighting competitions/기상 데이터/time_climate.csv"
climate_data_time = pd.read_csv(path, encoding = 'cp949')

### 적설량 추가

In [ ]:
# 일별 기상정보에 적설량 정보가 없음
print(climate_data_day["일 최심적설(cm)"].value_counts())
print(climate_data_day["일 최심신적설(cm)"].value_counts())
print(climate_data_day["합계 3시간 신적설(cm)"].value_counts())

Series([], Name: 일 최심적설(cm), dtype: int64)
Series([], Name: 일 최심신적설(cm), dtype: int64)
Series([], Name: 합계 3시간 신적설(cm), dtype: int64)


In [ ]:
# 시간별 기상정보에서의 일적설량 (cm)
DAY_MSNF_data = climate_data_time.groupby("일시")["적설(cm)"].sum()

In [ ]:
# 일별 기상정보와 일일 적설량 합치기
climate_data = pd.merge(climate_data_day, DAY_MSNF_data, on = "일시", how = "inner")

## 기상정보 추가

In [ ]:
# 기상정보 데이터 프레임 생성
climate_que = []
for i in climate_data.index:
    YMD = pd.to_datetime(str(climate_data.at[i, "일시"]), format='%Y-%m-%d')
    AVRG_TMPRT = climate_data.at[i, "평균기온(°C)"]
    DAY_RAINQTY = climate_data.at[i, "일강수량(mm)"]
    DAY_MSNF = climate_data.at[i, "적설(cm)"]
    AVRG_WS = climate_data.at[i, "평균 풍속(m/s)"]
    AVRG_HUMIDITY = climate_data.at[i, "평균 상대습도(%)"] 
    climate_que.append([YMD, int(AVRG_TMPRT), DAY_RAINQTY, int(DAY_MSNF), int(AVRG_WS), int(AVRG_HUMIDITY)])
climate_dict = pd.DataFrame(climate_que, columns = ["YMD", "AVRG_TMPRT", "DAY_RAINQTY", "DAY_MSNF", "AVRG_WS", "AVRG_HUMIDITY"])

In [ ]:
climate_dict

,YMD,AVRG_TMPRT,DAY_RAINQTY,DAY_MSNF,AVRG_WS,AVRG_HUMIDITY
0,2021-01-01,-6,NaN,6,0,71
1,2021-01-02,-4,NaN,0,1,53
2,2021-01-03,-5,NaN,0,0,50
3,2021-01-04,-3,0.0,0,0,50
4,2021-01-05,-3,0.0,0,1,46
...,...,...,...,...,...,...
360,2021-12-27,-7,0.0,0,0,59
361,2021-12-28,-3,NaN,0,0,72
362,2021-12-29,0,0.0,0,1,72
363,2021-12-30,-2,NaN,0,2,45


In [ ]:
# 비가 오지 않은날(결측치) 0으로 바꿔주기
climate_dict["DAY_RAINQTY"] = climate_dict["DAY_RAINQTY"].fillna(0).apply(lambda x : int(x))

In [ ]:
# 기상정보 DF 합치기
climate_dict.rename(columns = {"YMD":"OCRN_YMD"}, inplace =True)
climate_dict.OCRN_YMD=climate_dict.OCRN_YMD.astype(str) # type 변경
data_DF = pd.merge(data_DF, climate_dict, on = ["OCRN_YMD"], how="left").fillna(0)